In [1]:
# import packages
import numpy as np
import pandas as pd
import os

# Prepare Trips Data

In [10]:
# define dir
data_dir = "Data"

In [11]:
os.listdir(data_dir)

['trips_25_9.csv_processed.csv',
 'travelers.xlsx',
 'trips_23_9.csv_processed.csv',
 'trips_26_9.csv_processed.csv',
 'trips_21_9.csv_processed.csv',
 'trips_20_9.csv_processed.csv',
 'satisfaction.csv',
 'trips_22_9.csv_processed.csv',
 'incidents.csv',
 'trips_24_9.csv_processed.csv',
 'Data_Description.docx',
 'stations.csv',
 'stops.csv',
 'facilities.csv']

In [ ]:
# get all the infrabel trip files
all_trips = [obs for obs in os.listdir(data_dir) if ".DS" not in obs]

In [ ]:
all_trips

In [ ]:
full_trips = pd.DataFrame()
print(full_trips.shape)

for trip in all_trips:
    # import 
    df = pd.read_csv(os.path.join(data_dir, trip), sep=",")
    full_trips = full_trips.append(df)
    print(full_trips.shape)

# Import Data

## Facilities

In [ ]:
# import facilities
facilities = pd.read_csv("../data/facilities.csv")

In [ ]:
# check
facilities.head(5)

In [ ]:
# check number of missing values per variable
for col in facilities.columns:
    missings = len(facilities[col][facilities[col].isnull()]) / float(len(facilities))
    print(col, missings)

In [ ]:
#Check data type of columns
facilities.dtypes

In [ ]:
facilities['sales_open_monday2'] = pd.to_datetime(facilities['sales_open_monday'].astype(str), format='%H:%M')-pd.to_datetime('00:00', format='%H:%M')

In [ ]:
# see new column is of type timedelta64
facilities.dtypes

In [ ]:
# for instance only use subset of 'late openers'
late_openers = facilities[facilities['sales_open_monday2'] > pd.Timedelta(8,'h')]

In [ ]:
late_openers['sales_open_monday']

In [ ]:
# or use it to impute missing values
facilities['sales_open_monday2'].fillna((facilities['sales_open_monday2'].mean()), inplace=True)

In [ ]:
# check number of missing values per variable
for col in facilities.columns:
    missings = len(facilities[col][facilities[col].isnull()]) / float(len(facilities))
    print(col, missings)

## Travelers

In [ ]:
# import data
travelers = pd.read_excel("../data/travelers.xlsx", header=1, index_col=0)

In [ ]:
# check
travelers.head(5)

In [ ]:
# rename
travelers = travelers.rename({"Avg number of travelers in the week": "week",
                              "Avg number of travelers on Saturday": "saturday",
                              "Avg number of travelers on Sunday": "sunday"}, axis=1)

In [ ]:
# check number of missing values per variable
for col in travelers.columns:
    missings = len(travelers[col][travelers[col].isnull()]) / float(len(travelers))
    print(col, missings)

In [ ]:
# check missings
# change settings to visualize ALL rows
pd.set_option('display.max_rows', None)
print(travelers[travelers.isnull().any(axis=1)])

# change settings back
pd.reset_option('display.max_rows')

In [ ]:
# Interesting: never completely missing
# Inspection Wikipedia and NMBS website revealed no train rides on these dates for these stations (e.g., Baasrode-Zuid & Buda only train rides during the week)
# Therefore we impute every missing value with zero

travelers['week'].fillna(0, inplace=True)
travelers['saturday'].fillna(0, inplace=True)
travelers['sunday'].fillna(0, inplace=True)

In [ ]:
# create total
travelers["week_total"] = 5 * travelers["week"] + travelers["saturday"] + travelers["sunday"]

In [ ]:
# get weekend avg
travelers["weekend"] = (travelers["sunday"] + travelers["saturday"]) / float(2)

In [ ]:
# get avg travelers per day
travelers["avg_day"] = travelers["week_total"] / float(7)

In [ ]:
# check top 5 stations with highest number of travelers during the weekend
travelers.sort_values(by="week", ascending=False)[["Station", "week"]].head(5)

In [ ]:
# check top 5 stations with highest number of travelers during the week
travelers.sort_values(by="weekend", ascending=False)[["Station", "weekend"]].head(5)

Most remarkable differences are between Brussels Midi and Brussels North. North is in the middle of business centre ==> attracts many commuters during the week while Brussels Midi is the most important international railway station of Belgium and thus attracts many tourists. Also notice how Antwerpen and Leuven almost have equal travellers during the week and are off by almost a factor two during the weekend. This could signify a more or less equal commute potential but a far greater touristic potential for Antwerp. However, both are edjucated guesses based on my personal knowledge about the country. Implementing this mathematical and on a larger scale requires external data!

Other explanations for the commute numbers may possible lay in the number of facilities. As a proof of concept, let's try to mathematically proof whether weekly commute numbers are linked to availabilty of free parking and/or tram stations nearby. To do so, we will link the facilities and travelers datasets. We will impute missing values of free parking and tram with these facilities not being available. However, do note that this is not necessarily the best assumption!

In [ ]:
# impute columns with zero values
facilities['free_parking'].fillna(0, inplace=True)
facilities['tram'].fillna(0, inplace=True)

In [ ]:
facilities.shape

In [ ]:
travelers.shape

In [ ]:
# PROBLEM: no exact match in traveler/facilities information
# ASSUMPTION: travelers is subset of facilities

# convert to lower case
facilities['name'] = facilities['name'].str.lower()
travelers['Station'] =  travelers['Station'].str.lower()

In [ ]:
# check overlap
len(list(set(facilities['name']).intersection(set(travelers['Station']))))

In [ ]:
# around 80 which will need manual imputation
intersection = list(set(facilities['name']).intersection(set(travelers['Station'])))

still_needed = set(travelers['Station']).difference(intersection)

In [ ]:
len(still_needed)

In [ ]:
still_needed

In [ ]:
facility_names = set(facilities['name']).difference(intersection)

In [ ]:
facility_names

The facilities dateset also includes international stations, more small stations and different names for the stations with bilingual names or more complicated names. We will have to impute these manually. I will do some, but you will have to create a dictionary of all linked names in order to impute those names.

In [ ]:
# dictionary with correct names
Dict = dict({'antwerpen-caal': 'antwerpen-centraal', 
             'arcades': 'arcaden/arcades', 
             'beignee':'beignée', 
             'berchem-st-ag.-berchem':'sint-agatha-berchem/berchem-sainte-agathe'}) 

In [ ]:
# replace names
travelers = travelers.replace({"Station": Dict})

In [ ]:
# check if overlap +4 (previously overlap = 473)
len(list(set(facilities['name']).intersection(set(travelers['Station']))))

Overlap has increased by four. These four are the four stations I have mapped out.

Let's assume we have imputed all station names. Now we can merge the two datasets and run an analysis to see whether tram and free parking availability correlate to the number of travelers on a weekday. Do note that the relationship can be bi-directional. People can be inclined to use stations with connection to trams, but public transport companies can also be more likely to link their network with popular train stations.

In [ ]:
# first merge
merge = pd.merge(facilities, travelers, left_on='name', right_on='Station')

In [ ]:
# check if all were matched
merge.shape

To perform a multivariate regression, we will use the statsmodels package. Note that sklearn (which you will use later on in the course) also has an implementation of the model. However, sklearn focuses more on predictive performance (how good you can predict something) instead of statistical inference. Since we are interested whether relationships are significant, we will use statsmodels in this case.

In [ ]:
# only run this code once for installation of the package
# !pip3 install statsmodels

In [ ]:
import statsmodels.api as sm

# run multivariate regression
X = merge[['tram', 'free_parking']]
Y = merge['week']
X = sm.add_constant(X) # adding a constant: Y = beta0 + beta1*X1 + beta2*X2 + espilon instead of Y = beta1*X1 + beta2*X2 + epsilon
 
model = sm.OLS(Y, X).fit()
print_model = model.summary()

In [ ]:
# show results of regression
print_model

The model indicates both tram and free_parking as having a significant positive relationship with the number of weekly travelers, with tram access being of greater influence. The (adjusted) R² of slightly above 0.10 indicates a model which learns some useful relationships, but misses a lot of relevant control variables. However, we did not check the assumptions made by the multivariate linear regression model (e.g., multicollinearity), which could possibly violate the validity of our results. You should do these checks for your group assignment, as well as those for other tests/models.

### Group Assignment

Based on this week's course material, you should be able to solve following subquestions:
<br>
<br>
Q12: Which cities are the worst with regard to access to train facilities? You can do this by calculating the travel distance, travel time, ... Would you recommend based on this, and the visualization in Q5, to create some new routes?
<br>
<br>
Q14: Determine unique facilities that are very rare in orders of prevalence. Infer what may cause these facilities to be in place on their current locations. Are there possible other stations that could benefit from these facilities?
<br>
<br>
Q16: Is delay determined  by  possible delay  in the  previous  station?  (Hint: this is a form ofautocorrelation).
<br>
<br>
Q17: Regress the number of facilities to both the number of daily trains and number of dailytravelers. Do this using two univariate regressions and determine which covariate is theprimary driver for number of facilities, based on the adequate goodness-of-fit measure.
<br>
<br>
Of course, you can already start on other analyses outside the starting questions. Or on the pre-processing of your data in order to solve the other questions.



